In [13]:
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)



# G1

In [14]:
import numpy as np

def objective_function(x):
    x = np.array(x).reshape(-1, 13)
    f = 5 * np.sum(x[:, :4], axis=1) - 5 * np.sum(x[:, :4] ** 2, axis=1) - np.sum(x[:, 4:], axis=1)
    return f

def constraint_function(x):
    x = np.array(x).reshape(-1, 13)
    g1 = 2 * x[:, 0] + 2 * x[:, 1] + x[:, 9] + x[:, 10] - 10
    g2 = 2 * x[:, 0] + 2 * x[:, 2] + x[:, 9] + x[:, 11] - 10
    g3 = 2 * x[:, 1] + 2 * x[:, 2] + x[:, 10] + x[:, 11] - 10
    g4 = -8 * x[:, 0] + x[:, 9]
    g5 = -8 * x[:, 1] + x[:, 10]
    g6 = -8 * x[:, 2] + x[:, 11]
    g7 = -2 * x[:, 3] - x[:, 4] + x[:, 9]
    g8 = -2 * x[:, 5] - x[:, 6] + x[:, 10]
    g9 = -2 * x[:, 7] - x[:, 8] + x[:, 11]

    return np.vstack((g1, g2, g3, g4, g5, g6, g7, g8, g9))  

def constraints(x):

    return np.all(constraint_function(x) <= 0, axis=0)

def objective_function_with_constraints(x):
    if not constraints(x):
        return 0
    else:
        return objective_function(x)


In [15]:
def get_real_pop(): # populacao com todas as particulas válidas 
  filename = '/content/populacao_teste.txt'

  # Lista para armazenar os indivíduos lidos
  real_population = []

  # Ler o arquivo de texto
  with open(filename, 'r') as file:
      for line in file:
          individual = list(map(float, line.strip().split()))
          real_population.append(individual)
  real_population  = np.array(real_population)
  return real_population

## pso 

In [ ]:
import numpy as np
import pandas as pd

# Número de partículas
n_particles = 50

# Número de dimensões
n_dimensions = 13

# Limites inferiores e superiores para cada dimensão
lb = [0]*9 + [0]*3 + [0]
ub = [1]*9 + [100]*3 + [1]

# Parâmetros do PSO
c1 = 2
c2 = 2
w = 0.7
max_iter = 300
n_experiments = 30

# Armazenar resultados
results = []

pso_final = []
#PSO
for exp in range(n_experiments):
    # Inicializar enxame
    #particles = np.empty((n_particles, n_dimensions))
    #for i in range(n_particles):  ## gerar individuos válidos 
    #    while True:
    #        particle = np.random.uniform(low=lb, high=ub, size=n_dimensions)
    #        if objective_function_with_constraints(particle) != 0:
    #            particles[i] = particle
    #            break
    particles = get_real_pop()
    personal_best_scores = np.array([objective_function_with_constraints(p) for p in particles])

    velocities = np.zeros((n_particles, n_dimensions)) #inicializar velocidades com 0 
    personal_best_positions = particles.copy()

    # Inicializar o melhor global
    global_best_position = personal_best_positions[np.argmin(personal_best_scores)]
    global_best_score = min(personal_best_scores)

    # Loop principal
    for i in range(max_iter):
        print(f'------------------')
        print(f'experimento {exp+1} de {n_experiments}')
        print(f'iteração {i+1}')
        print(f'Global best score: {global_best_score}')
        for j in range(n_particles):
            # Atualizar velocidade
            velocities[j] = (w * velocities[j] +
                             c1 * np.random.rand() * (personal_best_positions[j] - particles[j]) +
                             c2 * np.random.rand() * (global_best_position - particles[j]))
            
            # Atualizar posição
            particles[j] += velocities[j]
            
            # Aplicar limites
            particles[j] = np.clip(particles[j], lb, ub)
            
            # Atualizar o melhor pessoal
            score = objective_function_with_constraints(particles[j])
            if score < personal_best_scores[j]:
                personal_best_positions[j] = particles[j].copy()
                personal_best_scores[j] = score
                
                # Atualizar o melhor global
                if score < global_best_score:
                    global_best_position = particles[j].copy()
                    global_best_score = score

        # Armazenar resultados
        results.append({
            'Experiment': exp+1,
            'Iteration': i+1,
            'Best Fitness': np.min(global_best_score),
            'Best Particle': global_best_position,
            'Worst Fitness': np.max(personal_best_scores),
            'Average Fitness': np.mean(personal_best_scores),
            'Standard Deviation': np.std(personal_best_scores),
            'Median Fitness': np.median(personal_best_scores),
        })
    pso_final.append(np.min(global_best_score))
        

# Criar dataframe com os resultados
df = pd.DataFrame(results)

df.head(5)



In [ ]:
df.head(1000)

In [ ]:
df_pso = df.sort_values('Best Fitness', ascending=True).reset_index()
best_PSO_individual = df_pso.loc[0, 'Best Particle']
fitness_best_PSO_individual = df_pso.loc[0, 'Best Fitness']
df_pso.head()

In [ ]:
best_PSO_individual


## ga

In [17]:
import math
import random
import numpy as np
import pandas as pd

# Definindo os parâmetros do algoritmo genético
pop_size = 50
genes = 13
generations = 300
lb = [0]*9 + [0]*3 + [0]
ub = [1]*9 + [100]*3 + [1]
crossover_prob = 0.9
mutation_prob = 0.12
n_experiments = 30

results_ga = []
ga_final = []

for exp in range(n_experiments):
   # Inicialização da população
   # Inicializar enxame
   #particles = np.empty((n_particles, n_dimensions))
   #for i in range(n_particles):  ## gerar individuos válidos 
   #    while True:
   #        particle = np.random.uniform(low=lb, high=ub, size=n_dimensions)
   #        if objective_function_with_constraints(particle) != 0:
   #            particles[i] = particle
   #            break
   #population = particles
    population = get_real_pop()
    for gen in range(generations):
        # Avaliação da população
        fitness_values = [objective_function_with_constraints(ind) for ind in population]
        print(f'------------------')
        print(f'experimento {exp+1} de {n_experiments}')
        print(f'generation {gen+1}')
        print(f'Global best score: {min(fitness_values)}')
        print(f'Global worst score: {max(fitness_values)}')

        # Seleção por torneio
        parents = []
        while len(parents) < len(population):
            contenders_indices = random.sample(range(len(population)), 2)
            contenders = [population[i] for i in contenders_indices]
            contenders_fitness = [fitness_values[i] for i in contenders_indices]
            best_contender = contenders[contenders_fitness.index(min(contenders_fitness))]
            parents.append(best_contender)

        # Garantindo que o número de pais seja par
        if len(parents) % 2 != 0:
            parents.pop()



        # Assegurando que o melhor indivíduo sempre estará na próxima geração
        best_individual = population[np.argmin(fitness_values)]

        # Crossover Uniforme
        offspring = []
        for i in range(0, len(parents), 2):
            parent1 = parents[i]
            parent2 = parents[i+1]
            child1 = parent1.copy()
            child2 = parent2.copy()
            for j in range(len(parent1)):
                if random.random() < crossover_prob:
                    child1[j] = parent2[j]
                    child2[j] = parent1[j]
            offspring.extend([child1, child2])
            
       # Crossover Blend
        alpha = 0.5
        for i in range(0, len(parents), 2):
            parent1 = np.array(parents[i])
            parent2 = np.array(parents[i+1])

            d = np.abs(parent1 - parent2)
            low = np.clip(np.minimum(parent1, parent2) - alpha * d, lb, ub)
            high = np.clip(np.maximum(parent1, parent2) + alpha * d, lb, ub)
            child1 = np.random.uniform(low, high)
            child2 = np.random.uniform(low, high)

            offspring.extend([child1.tolist(), child2.tolist()])
            

            


        # Substituição na população
        population = offspring + [best_individual]

        # Manter o tamanho da população constante em 50 indivíduos
        fitness_values = [objective_function_with_constraints(ind) for ind in population]
        pop_with_fitness = list(zip(population, fitness_values))
        pop_with_fitness.sort(key=lambda x: x[1])
        population, fitness_values = map(list, zip(*pop_with_fitness[:50]))


        # Mutação
        best_index = np.argmin(fitness_values)  
        for i in range(len(population)):
            if i != best_index and np.random.rand() <= mutation_prob:
                mutation_point = np.random.randint(genes)
                new_value = np.random.uniform(lb[mutation_point], ub[mutation_point])
                temp_individual = population[i].copy()
                temp_individual[mutation_point] = new_value
                if constraints(temp_individual):
                    population[i] = temp_individual

        # Reavaliação da população após crossover e mutação
        fitness_values = [objective_function_with_constraints(ind) for ind in population]

        best_fitness = min(fitness_values)
        worst_fitness = max(fitness_values)
        mean_fitness = np.mean(fitness_values)
        std_dev_fitness = np.std(fitness_values)

        results_ga.append({
            'experimento':exp+1,
            'geração': gen,
            'melhor_fitness': best_fitness,
            'melhor_individuo': population[np.argmin(fitness_values)],
            'pior_fitness': worst_fitness,
            'media_fitness': mean_fitness,
            'desvio_padrao': std_dev_fitness,
            'numero_individuos': len(population),
        })

         
    ga_final.append(best_fitness)   

df_ga = pd.DataFrame(results_ga)
df_ga.head(5)




Streaming output truncated to the last 5000 lines.
------------------
experimento 27 de 30
generation 201
Global best score: [-14.03641951]
Global worst score: [-12.79929464]
------------------
experimento 27 de 30
generation 202
Global best score: [-14.03641975]
Global worst score: [-13.14910977]
------------------
experimento 27 de 30
generation 203
Global best score: [-14.0364202]
Global worst score: [-13.9073968]
------------------
experimento 27 de 30
generation 204
Global best score: [-14.03642054]
Global worst score: [-14.03641953]
------------------
experimento 27 de 30
generation 205
Global best score: [-14.03642159]
Global worst score: [-14.03641998]
------------------
experimento 27 de 30
generation 206
Global best score: [-14.03642222]
Global worst score: [-12.804432]
------------------
experimento 27 de 30
generation 207
Global best score: [-14.03642261]
Global worst score: [-13.44795282]
------------------
experimento 27 de 30
generation 208
Global best score: [-14.036423

,experimento,geração,melhor_fitness,melhor_individuo,pior_fitness,media_fitness,desvio_padrao,numero_individuos
0,1,0,[-5.687991725871292],"[0.2148786318055651, 0.9827814502287309, 0.214...",[-2.1676474354242448],-3.770011,0.819700,50
1,1,1,[-7.281937443244109],"[0.9435821969473398, 0.9625511041703319, 0.219...",[-3.515634311884626],-4.626318,0.836230,50
2,1,2,[-7.281937443244109],"[0.9435821969473398, 0.9625511041703319, 0.219...",[-4.664600855093896],-5.727711,0.652505,50
3,1,3,[-7.705925335905453],"[0.9751765630242231, 0.9721210569314125, 0.328...",[-5.6250758419298315],-6.270640,0.502779,50
4,1,4,[-8.29182552740962],"[0.4743012701823079, 0.9959878594285674, 0.580...",[-5.543320040873541],-6.811453,0.543391,50


In [ ]:
df_ga.head(1000)

In [ ]:
df_ga = df_ga.sort_values('melhor_fitness', ascending=True).reset_index()
best_ga_individual = df_ga.loc[0, 'melhor_individuo']
fitness_best_ga_individual = df_ga.loc[0, 'melhor_fitness']
df_ga.head()

## de

In [ ]:
# Definindo os parâmetros do de
pop_size = 50
genes = 13
generations = 300
lb = [0]*9 + [0]*3 + [0]
ub = [1]*9 + [100]*3 + [1]
crossover_prob = 0.6
mutation_prob = 0.1
F = 0.8  # Fator de escala para a mutação

results_de = []
de_final = []

for exp in range(30):
   # Inicialização da população
   # Inicializar enxame
   #particles = np.empty((n_particles, n_dimensions))
   #for i in range(n_particles):  ## gerar individuos válidos 
   #    while True:
   #        particle = np.random.uniform(low=lb, high=ub, size=n_dimensions)
   #        if objective_function_with_constraints(particle) != 0:
   #            particles[i] = particle
   #            break
   #population = particles
    population = get_real_pop()

    # Loop das gerações
    for gen in range(generations):
        # Avaliação da população
        fitness_values = [objective_function_with_constraints(ind) for ind in population]

        # Salvar o melhor indivíduo para elitismo
        best_index = np.argmin(fitness_values)
        best_individual = population[best_index].copy()

        # Evolução Diferencial
        new_population = population.copy()
        for i in range(pop_size):
            indices = [idx for idx in range(pop_size) if idx != i]
            a, b, c = population[np.random.choice(indices, 3, replace=False)]
            mutant = np.clip(a + F * (b - c), lb, ub)  # Mutação
            cross_points = np.random.rand(genes) < crossover_prob
            if not np.any(cross_points):
                cross_points[np.random.randint(0, genes)] = True
            trial = np.where(cross_points, mutant, population[i])  # Crossover
            if objective_function_with_constraints(trial) < fitness_values[i]:  # Aceitação
                new_population[i] = trial

        population = new_population

        # Elitismo: garantir que o melhor indivíduo esteja na nova população
        new_fitness_values = [objective_function_with_constraints(ind) for ind in population]
        worst_index = np.argmax(new_fitness_values)
        if new_fitness_values[worst_index] > fitness_values[best_index]:
            population[worst_index] = best_individual

        # Estatísticas da geração
        fitness_values = [objective_function_with_constraints(ind) for ind in population]
        best_fitness = min(fitness_values)
        worst_fitness = max(fitness_values)
        mean_fitness = np.mean(fitness_values)
        std_dev_fitness = np.std(fitness_values)

        results_de.append({
            'experimento': exp+1,
            'geracao': gen,
            'melhor_fitness': best_fitness,
            'melhor_particula': population[np.argmin(fitness_values)],
            'pior_fitness': worst_fitness,
            'media_dos_fitness': mean_fitness,
            'desvio_padrao': std_dev_fitness,
            'n_de_individuos': len(population),
        })
        print(f'Experimento {exp+1}, Geração {gen+1}')
        print(f'Melhor partícula: {population[np.argmin(fitness_values)]}, Fitness: {best_fitness}')
        print(f'Pior partícula: {population[np.argmax(fitness_values)]}, Fitness: {worst_fitness}')
        print('---'*20)
    de_final.append(best_fitness)

df_de = pd.DataFrame(results_de)
df_de.head(5)


In [ ]:
df_de.head(1000)

In [ ]:
df_de = df_de.sort_values('melhor_fitness', ascending=True).reset_index()
best_de_individual = df_de.loc[0, 'melhor_particula']
fitness_best_de_individual = df_de.loc[0, 'melhor_fitness']
df_de.head()

## ES

#parametros globais + EE-CMA

In [26]:
import numpy as np
from scipy.linalg import sqrtm

# Definindo os parâmetros da evolução estratégica
pop_size = 50
genes = 13
generations = 300
lb = [0]*9 + [0]*3 + [0]
ub = [1]*9 + [100]*3 + [1]
sigma = 0.1  # Desvio padrão para a mutação
tau = 1/np.sqrt(2*genes)  # Parâmetro de autoadaptação

results_es = []
es_g_final = []



# Loop dos experimentos
for exp in range(30):
  population = get_real_pop()
  sigmas = sigma * np.ones((pop_size, genes, genes))
  # Loop das gerações
  # Loop das gerações
  for gen in range(generations):
      # Avaliação da população
      fitness_values = [objective_function_with_constraints(ind) for ind in population]

      # Elitismo: guardar o melhor indivíduo
      best_index = np.argmin(fitness_values)
      best_individual = population[best_index].copy()

      # Estratégia Evolutiva
      for i in range(pop_size):
          # Seleção de pais aleatórios
          parent_indices = np.random.choice(pop_size, 2, replace=False)
          parents = population[parent_indices]

          # CMA-ES
          mean = np.mean(parents, axis=0)
          cov = np.cov(parents.T)
          child = np.random.multivariate_normal(mean, cov)
          child = np.clip(child, lb, ub)

          # Adição de mutação
          child += sigma * np.random.randn(genes)
          child = np.clip(child, lb, ub)

          # Seleção (mu, sigma)
          if objective_function_with_constraints(child) < fitness_values[i]:
              population[i] = child

      # Elitismo: substituir o pior indivíduo pelo melhor da geração anterior, se necessário
      fitness_values = [objective_function_with_constraints(ind) for ind in population]
      worst_index = np.argmax(fitness_values)
      if fitness_values[worst_index] > objective_function_with_constraints(best_individual):
          population[worst_index] = best_individual

      # Estatísticas da geração
      best_fitness = min(fitness_values)
      worst_fitness = max(fitness_values)
      mean_fitness = np.mean(fitness_values)
      std_dev_fitness = np.std(fitness_values)

      results_es.append({
          'experimento': exp+1,
          'geracao': gen,
          'melhor_fitness': best_fitness,
          'melhor_particula': population[np.argmin(fitness_values)],
          'pior_fitness': worst_fitness,
          'media_dos_fitness': mean_fitness,
          'desvio_padrao': std_dev_fitness,
          'n_de_individuos': len(population),
      })

      # Imprime as melhores e piores partículas
      print(f'Experimento {exp+1}, Geração {gen+1}')
      print(f'Melhor partícula: {population[np.argmin(fitness_values)]}, Fitness: {best_fitness}')
      print(f'Pior partícula: {population[np.argmax(fitness_values)]}, Fitness: {worst_fitness}')
      print('---'*20)
      
  es_g_final.append(best_fitness)

df_g_es = pd.DataFrame(results_es)
df_g_es.head(5)





/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = getattr(asarray(obj), method)(*args, **kwds)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:163: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = asanyarray(a)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:197: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If

Streaming output truncated to the last 5000 lines.
Experimento 6, Geração 65
Melhor partícula: [0.9492508  1.         1.         1.         0.9874932  0.85701776
 0.94633519 0.98191668 0.97475037 2.72527431 1.96624069 2.82767595
 0.99499928], Fitness: [-13.02083483]
Pior partícula: [1.         0.99685082 0.9976408  1.         1.         1.
 0.76345564 0.98061566 1.         2.31060562 2.233413   2.75633176
 1.        ], Fitness: [-12.11342311]
------------------------------------------------------------
Experimento 6, Geração 66
Melhor partícula: [0.9492508  1.         1.         1.         0.9874932  0.85701776
 0.94633519 0.98191668 0.97475037 2.72527431 1.96624069 2.82767595
 0.99499928], Fitness: [-13.02083483]
Pior partícula: [0.9492508  1.         1.         1.         0.9874932  0.85701776
 0.94633519 0.98191668 0.97475037 2.72527431 1.96624069 2.82767595
 0.99499928], Fitness: [-12.18245148]
------------------------------------------------------------
Experimento 6, Geração 67
M

KeyboardInterrupt: ignored

In [24]:
df_g_es.head(1000)

,experimento,geracao,melhor_fitness,melhor_particula,pior_fitness,media_dos_fitness,desvio_padrao,n_de_individuos
0,1,0,[-7.038317794267071],"[1.0, 1.0, 1.0, 1.0, 0.9767041258423673, 0.981...",[-0.4730548589364325],-3.021819,1.53229,50
1,1,1,[-7.038317794267071],"[1.0, 1.0, 1.0, 1.0, 0.9767041258423673, 0.981...",[-0.5681295453329813],-3.839351,1.30165,50
2,1,2,[-7.038317794267071],"[1.0, 1.0, 1.0, 1.0, 0.9767041258423673, 0.981...",[-2.5328744634655473],-4.572442,1.031709,50
3,1,3,[-7.038317794267071],"[1.0, 1.0, 1.0, 1.0, 1.0, 0.9618622956334919, ...",[-2.8226345409119586],-5.035638,0.934847,50
4,1,4,[-8.212463925736483],"[1.0, 1.0, 0.9988373150051861, 1.0, 1.0, 1.0, ...",[-3.757688739937672],-5.510251,0.998011,50
5,1,5,[-8.212463925736483],"[1.0, 1.0, 0.9988373150051861, 1.0, 1.0, 1.0, ...",[-4.397628751978377],-5.939375,0.922722,50
6,1,6,[-8.212463925736483],"[1.0, 1.0, 0.9988373150051861, 1.0, 1.0, 1.0, ...",[-4.421297723662704],-6.402352,0.897781,50
7,1,7,[-8.212463925736483],"[1.0, 1.0, 0.9988373150051861, 1.0, 1.0, 1.0, ...",[-4.471027901184785],-6.599786,0.860331,50
8,1,8,[-8.795340899753777],"[0.9931436014647279, 1.0, 1.0, 1.0, 1.0, 1.0, ...",[-4.504972304966477],-6.971295,0.832869,50
9,1,9,[-8.795340899753777],"[0.9931436014647279, 1.0, 1.0, 1.0, 1.0, 1.0, ...",[-5.735426436710842],-7.329137,0.617222,50


In [25]:
df_g_es = df_g_es.sort_values('melhor_fitness', ascending=True).reset_index()
best_g_es_individual = df_g_es.loc[0, 'melhor_particula']
fitness_best_es_individual = df_g_es.loc[0, 'melhor_fitness']
df_g_es.head()

,index,experimento,geracao,melhor_fitness,melhor_particula,pior_fitness,media_dos_fitness,desvio_padrao,n_de_individuos
0,891,3,291,[-14.963308806428719],"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",[-14.779935909361575],-14.808806,0.045865,50
1,890,3,290,[-14.963308806428719],"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",[-14.779935909361575],-14.805139,0.040366,50
2,898,3,298,[-14.963308806428719],"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",[-14.792358697668671],-14.832988,0.069366,50
3,897,3,297,[-14.963308806428719],"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",[-14.792358697668671],-14.829569,0.067032,50
4,896,3,296,[-14.963308806428719],"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",[-14.792358697668671],-14.82615,0.064433,50


## resultados obtidos

In [ ]:
resultados_G1 = pd.DataFrame({
    'EVO. Alg.':['PSO','GA','DE','ES'],
    'Best Fitness':[fitness_best_PSO_individual,fitness_best_ga_individual,fitness_best_de_individual,fitness_best_es_individual],
    'Best Particle EA':[best_PSO_individual,best_ga_individual,best_de_individual,best_es_individual]})

resultados_G1 = resultados_G1.sort_values('Best Fitness',ascending=True).reset_index()
resultados_G1.head()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt


data = pd.DataFrame({
    'PSO': pso_final,
    'GA': ga_final,
    'DE': de_final,
    'ES': es_final,
})


plt.figure(figsize=(10, 6))
sns.boxplot(data=data)
plt.title('Boxplot dos Fitness Finais')
plt.ylabel('Fitness')
plt.show()


# G2

## ES

## resultados obtidos